In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from string import ascii_letters
from rapidfuzz import process, fuzz
import matplotlib.pyplot as plt
import datetime as dt
import requests
from lxml import html
import math
import re
import json
import os

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)


In [3]:
raw_ld_df = pd.read_csv("data/raw_loan_data_date.csv", low_memory = False)
raw_ld_df.head()

,BORROWER_NAME,CITY,STATE,ZIP,BANK,BANK_STATE,INDUSTRY_ID,APPROVAL_DATE,APPROVAL_YEAR,TERM,EMPLOYEE_COUNT,IS_NEW,JOBS_CREATED_COUNT,JOBS_RETAINED_COUNT,FRANCHISE_CODE,IS_URBAN,IS_REVOLVER,IS_LOW_DOC,DEFAULT_DATE,DISBURSEMENT_DATE,DISBURSEMENT_AMOUNT,BALANCE_AMOUNT,LOAN_STATUS,CHARGE_OFF_AMOUNT,LOAN_AMOUNT,SBA_APPROVED_AMOUNT,TREASURY_YIELD,CPI_INDEX,GDP,MORTGAGE_30_US_FIXED,UNRATE,INDPRO_INDEX,UMCSENT_INDEX,CSUSHPINSA_INDEX,CP_INDEX,FEDFUNDS_RATE
0,SHANGRI LA HOTEL INC,ELLENVILLE N Y,NY,12428,BANK OF AMERICA NATL ASSOC,NY,0,7-Dec-61,1962,300,0,0.0,0,0,0,0,N,N,16-Feb-90,NaN,$0.00,$0.00,NaN,"$2,613.00","$100,000.00","$90,000.00",3.945833,30.253333,603.92150,NaN,5.566667,25.332875,95.475,NaN,35.90825,2.708333
1,CATSKILL MILLS INC,WALTON N Y,NY,13856,"NBT BANK, NATIONAL ASSOCIATION",NY,0,27-Nov-64,1965,252,0,0.0,0,0,0,0,N,N,3-Apr-89,NaN,$0.00,$0.00,NaN,"$55,625.00","$150,000.00","$127,500.00",4.282500,31.528333,742.29025,NaN,4.508333,31.546817,103.425,NaN,53.34175,4.075000
2,TRYON COATS & LEATHER,JOHNSTOWN NY,NaN,0,KEYBANK NATIONAL ASSOCIATION,NY,0,18-May-66,1966,282,0,0.0,0,0,0,0,N,N,29-Mar-90,16-Aug-66,"$60,000.00",$0.00,CHGOFF,"$6,084.00","$60,000.00","$54,000.00",4.923333,32.470833,813.41375,NaN,3.791667,34.337933,93.800,NaN,58.15200,5.111667
3,LOUIS GELBER & ELSIE RODRIGUEZ,BROOKLYN,NY,11029,GOVERNMENT DEVEL BK FOR PUERTO,PR,0,28-Feb-67,1967,72,0,0.0,0,0,0,0,N,N,19-May-92,NaN,$0.00,$0.00,NaN,"$8,023.00","$30,000.00","$15,000.00",5.073333,33.375000,859.95625,NaN,3.841667,35.079150,94.975,NaN,56.61175,4.220000
4,CHARLENE R PAYNE,BOGALUSA,LA,70427,WHITNEY BANK,MS,0,25-Apr-67,1967,228,4,0.0,0,0,0,0,N,N,15-Mar-89,NaN,$0.00,$0.00,NaN,"$9,915.00","$45,000.00","$40,500.00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
unprocessed_ld_df = raw_ld_df.copy()
unprocessed_ld_df.dtypes


BORROWER_NAME        object
CITY                 object
STATE                object
ZIP                   int64
BANK                 object
                     ...   
INDPRO_INDEX        float64
UMCSENT_INDEX       float64
CSUSHPINSA_INDEX    float64
CP_INDEX            float64
FEDFUNDS_RATE       float64
Length: 36, dtype: object

In [5]:
def adjust_year(dt):
    return dt.replace(year=dt.year - 100) if dt.year >= 2025 else dt

date_format = "%d-%b-%y"
date_cols = ["APPROVAL_DATE", "DISBURSEMENT_DATE", "DEFAULT_DATE"]
for col in date_cols:
    unprocessed_ld_df[col] = unprocessed_ld_df[col].astype("string")
    unprocessed_ld_df[col] = pd.to_datetime(unprocessed_ld_df[col], format=date_format)
    unprocessed_ld_df[col] = unprocessed_ld_df[col].apply(adjust_year)
unprocessed_ld_df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 36 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   BORROWER_NAME         899150 non-null  object        
 1   CITY                  899134 non-null  object        
 2   STATE                 899150 non-null  object        
 3   ZIP                   899164 non-null  int64         
 4   BANK                  897605 non-null  object        
 5   BANK_STATE            897598 non-null  object        
 6   INDUSTRY_ID           899164 non-null  int64         
 7   APPROVAL_DATE         899164 non-null  datetime64[ns]
 8   APPROVAL_YEAR         899164 non-null  object        
 9   TERM                  899164 non-null  int64         
 10  EMPLOYEE_COUNT        899164 non-null  int64         
 11  IS_NEW                899028 non-null  float64       
 12  JOBS_CREATED_COUNT    899164 non-null  int64         
 13 

In [6]:

unprocessed_ld_df[['DISBURSEMENT_AMOUNT', 'CHARGE_OFF_AMOUNT', 'LOAN_AMOUNT', 'SBA_APPROVED_AMOUNT']] = unprocessed_ld_df[['DISBURSEMENT_AMOUNT', 'CHARGE_OFF_AMOUNT', 'LOAN_AMOUNT', 'SBA_APPROVED_AMOUNT']].replace('[\\$,]', '', regex=True).astype(float)

In [7]:
unprocessed_ld_df['IS_REVOLVER'] = unprocessed_ld_df['IS_REVOLVER'].map({'Y': True, 'N': False})
unprocessed_ld_df['IS_LOW_DOC'] = unprocessed_ld_df['IS_LOW_DOC'].map({'Y': True, 'N': False})
unprocessed_ld_df.head()


,BORROWER_NAME,CITY,STATE,ZIP,BANK,BANK_STATE,INDUSTRY_ID,APPROVAL_DATE,APPROVAL_YEAR,TERM,EMPLOYEE_COUNT,IS_NEW,JOBS_CREATED_COUNT,JOBS_RETAINED_COUNT,FRANCHISE_CODE,IS_URBAN,IS_REVOLVER,IS_LOW_DOC,DEFAULT_DATE,DISBURSEMENT_DATE,DISBURSEMENT_AMOUNT,BALANCE_AMOUNT,LOAN_STATUS,CHARGE_OFF_AMOUNT,LOAN_AMOUNT,SBA_APPROVED_AMOUNT,TREASURY_YIELD,CPI_INDEX,GDP,MORTGAGE_30_US_FIXED,UNRATE,INDPRO_INDEX,UMCSENT_INDEX,CSUSHPINSA_INDEX,CP_INDEX,FEDFUNDS_RATE
0,SHANGRI LA HOTEL INC,ELLENVILLE N Y,NY,12428,BANK OF AMERICA NATL ASSOC,NY,0,1961-12-07,1962,300,0,0.0,0,0,0,0,False,False,1990-02-16,NaT,0.0,$0.00,NaN,2613.0,100000.0,90000.0,3.945833,30.253333,603.92150,NaN,5.566667,25.332875,95.475,NaN,35.90825,2.708333
1,CATSKILL MILLS INC,WALTON N Y,NY,13856,"NBT BANK, NATIONAL ASSOCIATION",NY,0,1964-11-27,1965,252,0,0.0,0,0,0,0,False,False,1989-04-03,NaT,0.0,$0.00,NaN,55625.0,150000.0,127500.0,4.282500,31.528333,742.29025,NaN,4.508333,31.546817,103.425,NaN,53.34175,4.075000
2,TRYON COATS & LEATHER,JOHNSTOWN NY,NaN,0,KEYBANK NATIONAL ASSOCIATION,NY,0,1966-05-18,1966,282,0,0.0,0,0,0,0,False,False,1990-03-29,1966-08-16,60000.0,$0.00,CHGOFF,6084.0,60000.0,54000.0,4.923333,32.470833,813.41375,NaN,3.791667,34.337933,93.800,NaN,58.15200,5.111667
3,LOUIS GELBER & ELSIE RODRIGUEZ,BROOKLYN,NY,11029,GOVERNMENT DEVEL BK FOR PUERTO,PR,0,1967-02-28,1967,72,0,0.0,0,0,0,0,False,False,1992-05-19,NaT,0.0,$0.00,NaN,8023.0,30000.0,15000.0,5.073333,33.375000,859.95625,NaN,3.841667,35.079150,94.975,NaN,56.61175,4.220000
4,CHARLENE R PAYNE,BOGALUSA,LA,70427,WHITNEY BANK,MS,0,1967-04-25,1967,228,4,0.0,0,0,0,0,False,False,1989-03-15,NaT,0.0,$0.00,NaN,9915.0,45000.0,40500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
is_new_counts = unprocessed_ld_df['IS_NEW'].value_counts(dropna=False)
print("Frequency distribution of 'IS_NEW':")
print(is_new_counts)

is_urban_counts = unprocessed_ld_df['IS_URBAN'].value_counts(dropna=False)
print("\nFrequency distribution of 'IS_URBAN':")
print(is_urban_counts)

Frequency distribution of 'IS_NEW':
IS_NEW
1.0    644869
2.0    253125
0.0      1034
NaN       136
Name: count, dtype: int64

Frequency distribution of 'IS_URBAN':
IS_URBAN
1    470654
0    323167
2    105343
Name: count, dtype: int64


In [9]:
# Knowing this info above, we can assume the frequency distribution of urban loans and new loans to be more likely, meaning 1 is boolean true
unprocessed_ld_df['IS_NEW'] = unprocessed_ld_df['IS_NEW'].map({1.0: True, 2.0: False})
unprocessed_ld_df['IS_URBAN'] = unprocessed_ld_df['IS_URBAN'].map({1: True, 2: False})


In [10]:
unprocessed_ld_df['LOAN_STATUS'] = unprocessed_ld_df['LOAN_STATUS'].map({"CHGOFF": False, "P I F": True})


In [11]:
unprocessed_ld_df.head(10000000)

,BORROWER_NAME,CITY,STATE,ZIP,BANK,BANK_STATE,INDUSTRY_ID,APPROVAL_DATE,APPROVAL_YEAR,TERM,EMPLOYEE_COUNT,IS_NEW,JOBS_CREATED_COUNT,JOBS_RETAINED_COUNT,FRANCHISE_CODE,IS_URBAN,IS_REVOLVER,IS_LOW_DOC,DEFAULT_DATE,DISBURSEMENT_DATE,DISBURSEMENT_AMOUNT,BALANCE_AMOUNT,LOAN_STATUS,CHARGE_OFF_AMOUNT,LOAN_AMOUNT,SBA_APPROVED_AMOUNT,TREASURY_YIELD,CPI_INDEX,GDP,MORTGAGE_30_US_FIXED,UNRATE,INDPRO_INDEX,UMCSENT_INDEX,CSUSHPINSA_INDEX,CP_INDEX,FEDFUNDS_RATE
0,SHANGRI LA HOTEL INC,ELLENVILLE N Y,NY,12428,BANK OF AMERICA NATL ASSOC,NY,0,1961-12-07,1962,300,0,NaN,0,0,0,NaN,False,False,1990-02-16,NaT,0.0,$0.00,NaN,2613.0,100000.0,90000.0,3.945833,30.253333,603.92150,NaN,5.566667,25.332875,95.475,NaN,35.90825,2.708333
1,CATSKILL MILLS INC,WALTON N Y,NY,13856,"NBT BANK, NATIONAL ASSOCIATION",NY,0,1964-11-27,1965,252,0,NaN,0,0,0,NaN,False,False,1989-04-03,NaT,0.0,$0.00,NaN,55625.0,150000.0,127500.0,4.282500,31.528333,742.29025,NaN,4.508333,31.546817,103.425,NaN,53.34175,4.075000
2,TRYON COATS & LEATHER,JOHNSTOWN NY,NaN,0,KEYBANK NATIONAL ASSOCIATION,NY,0,1966-05-18,1966,282,0,NaN,0,0,0,NaN,False,False,1990-03-29,1966-08-16,60000.0,$0.00,False,6084.0,60000.0,54000.0,4.923333,32.470833,813.41375,NaN,3.791667,34.337933,93.800,NaN,58.15200,5.111667
3,LOUIS GELBER & ELSIE RODRIGUEZ,BROOKLYN,NY,11029,GOVERNMENT DEVEL BK FOR PUERTO,PR,0,1967-02-28,1967,72,0,NaN,0,0,0,NaN,False,False,1992-05-19,NaT,0.0,$0.00,NaN,8023.0,30000.0,15000.0,5.073333,33.375000,859.95625,NaN,3.841667,35.079150,94.975,NaN,56.61175,4.220000
4,CHARLENE R PAYNE,BOGALUSA,LA,70427,WHITNEY BANK,MS,0,1967-04-25,1967,228,4,NaN,0,0,0,NaN,False,False,1989-03-15,NaT,0.0,$0.00,NaN,9915.0,45000.0,40500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,MARK DEMATTIA INCORPORATED,EWA BEACH,HI,96706,CENTRAL PACIFIC BANK,HI,492210,2014-06-12,2014,60,0,False,0,0,0,True,False,False,NaT,2014-06-12,50000.0,$0.00,True,0.0,50000.0,25000.0,2.540833,236.715000,17608.13825,4.168868,6.158333,102.305642,84.125,164.674667,1947.36300,0.089167
899160,PURSUIT SPORTS MEDICINE LLC,PORTLAND,OR,97232,U.S. BANK NATIONAL ASSOCIATION,OH,621111,2014-06-18,2014,60,1,True,0,1,0,True,False,False,NaT,2014-06-18,165000.0,$0.00,True,0.0,165000.0,82500.0,2.540833,236.715000,17608.13825,4.168868,6.158333,102.305642,84.125,164.674667,1947.36300,0.089167
899161,"Face Reality, Inc., dba Face R",San Leandro,CA,94577,CITY NATIONAL BANK,CA,325412,2014-06-19,2014,84,15,True,3,0,0,True,False,False,NaT,2014-06-19,150000.0,$0.00,True,0.0,150000.0,75000.0,2.540833,236.715000,17608.13825,4.168868,6.158333,102.305642,84.125,164.674667,1947.36300,0.089167
899162,HARDRIVES CONSTRUCTION INC,BILLINGS,MT,59103,BANK OF BRIDGER NATL ASSOC,MT,237310,2014-06-23,2014,6,30,True,0,30,0,False,True,False,NaT,2014-06-23,210000.0,$0.00,True,0.0,210000.0,157500.0,2.540833,236.715000,17608.13825,4.168868,6.158333,102.305642,84.125,164.674667,1947.36300,0.089167


In [12]:
processed_noReduced_ld_df = unprocessed_ld_df.copy()

In [15]:
economic_indicators = [
    'TREASURY_YIELD', 'CPI_INDEX', 'GDP', 'MORTGAGE_30_US_FIXED',
    'UNRATE', 'INDPRO_INDEX', 'UMCSENT_INDEX', 'CSUSHPINSA_INDEX',
    'CP_INDEX', 'FEDFUNDS_RATE'
]
processed_noReduced_ld_df = processed_noReduced_ld_df.dropna(subset=economic_indicators, how='all')
processed_noReduced_ld_df = processed_noReduced_ld_df[processed_noReduced_ld_df['APPROVAL_DATE'].dt.year >= 1990]
processed_noReduced_ld_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 737405 entries, 31172 to 899163
Data columns (total 36 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   BORROWER_NAME         737391 non-null  object        
 1   CITY                  737405 non-null  object        
 2   STATE                 737401 non-null  object        
 3   ZIP                   737405 non-null  int64         
 4   BANK                  736565 non-null  object        
 5   BANK_STATE            736560 non-null  object        
 6   INDUSTRY_ID           737405 non-null  int64         
 7   APPROVAL_DATE         737405 non-null  datetime64[ns]
 8   APPROVAL_YEAR         737405 non-null  object        
 9   TERM                  737405 non-null  int64         
 10  EMPLOYEE_COUNT        737405 non-null  int64         
 11  IS_NEW                736473 non-null  object        
 12  JOBS_CREATED_COUNT    737405 non-null  int64         
 13  

In [145]:


processed_noReduced_ld_df = processed_noReduced_ld_df.dropna(subset=['INDUSTRY_ID'])

processed_noReduced_ld_df = processed_noReduced_ld_df.dropna(subset=['INDUSTRY_ID',
                                                                     'IS_NEW',
                                                                     'IS_URBAN',
                                                                     'LOAN_STATUS',
                                                                     'IS_REVOLVER',
                                                                     'BANK',
                                                                     'BORROWER_NAME', 
                                                                     'IS_LOW_DOC',
                                                                     'DISBURSEMENT_DATE'])

processed_noReduced_ld_df = processed_noReduced_ld_df[processed_noReduced_ld_df['DISBURSEMENT_AMOUNT'] != 0]

processed_noReduced_ld_df['APPROVAL_DATE'] = pd.to_datetime(processed_noReduced_ld_df['APPROVAL_DATE'], errors='coerce')
processed_noReduced_ld_df.head()

,BORROWER_NAME,CITY,STATE,ZIP,BANK,BANK_STATE,INDUSTRY_ID,APPROVAL_DATE,APPROVAL_YEAR,TERM,EMPLOYEE_COUNT,IS_NEW,JOBS_CREATED_COUNT,JOBS_RETAINED_COUNT,FRANCHISE_CODE,IS_URBAN,IS_REVOLVER,IS_LOW_DOC,DEFAULT_DATE,DISBURSEMENT_DATE,DISBURSEMENT_AMOUNT,BALANCE_AMOUNT,LOAN_STATUS,CHARGE_OFF_AMOUNT,LOAN_AMOUNT,SBA_APPROVED_AMOUNT,TREASURY_YIELD,CPI_INDEX,GDP,MORTGAGE_30_US_FIXED,UNRATE,INDPRO_INDEX,UMCSENT_INDEX,CSUSHPINSA_INDEX,CP_INDEX,FEDFUNDS_RATE
119522,JOHNSSON REESE FINANCIAL SOLUT,CHICAGO,IL,60622,FIFTH THIRD BANK,IL,0,1994-05-17,1994,36,16,True,0,0,1,True,False,False,NaT,1994-10-31,20000.0,$0.00,True,0.0,20000.0,18000.0,7.08,148.225,7287.2365,8.380769,6.1,68.713433,92.266667,79.469083,438.35075,4.201667
119527,COMPUTERLAND OF DOWNERS GROVE,CHICAGO,IL,60515,BMO HARRIS BK NATL ASSOC,IL,0,1994-05-17,1994,84,6,True,0,0,19110,True,False,False,NaT,1994-10-31,250000.0,$0.00,True,0.0,250000.0,200000.0,7.08,148.225,7287.2365,8.380769,6.1,68.713433,92.266667,79.469083,438.35075,4.201667
119675,SEARS RETAIL DEALER STORE,SELMA,AL,36701,TRUSTMARK NATIONAL BANK,AL,0,1994-05-19,1994,120,3,False,0,0,1,False,False,False,NaT,1994-07-31,213500.0,$0.00,True,0.0,213500.0,181475.0,7.08,148.225,7287.2365,8.380769,6.1,68.713433,92.266667,79.469083,438.35075,4.201667
120580,NICOLE'S,CHICAGO,IL,60622,BANK OF AMERICA NATL ASSOC,IL,0,1994-05-31,1994,84,6,True,0,0,1,True,False,False,NaT,1994-07-31,75000.0,$0.00,True,0.0,75000.0,67500.0,7.08,148.225,7287.2365,8.380769,6.1,68.713433,92.266667,79.469083,438.35075,4.201667
121009,"KID'S TIME, INC.",SKOKIE,IL,60618,BANK OF AMERICA NATL ASSOC,IL,624410,1994-06-03,1994,84,1,False,0,0,45499,True,False,False,NaT,1994-10-31,60000.0,$0.00,True,0.0,60000.0,54000.0,7.08,148.225,7287.2365,8.380769,6.1,68.713433,92.266667,79.469083,438.35075,4.201667


In [146]:
processed_noReduced_ld_df = processed_noReduced_ld_df[processed_noReduced_ld_df['DISBURSEMENT_AMOUNT'] != 0]
processed_noReduced_ld_df.head(1000000000000000)

,BORROWER_NAME,CITY,STATE,ZIP,BANK,BANK_STATE,INDUSTRY_ID,APPROVAL_DATE,APPROVAL_YEAR,TERM,EMPLOYEE_COUNT,IS_NEW,JOBS_CREATED_COUNT,JOBS_RETAINED_COUNT,FRANCHISE_CODE,IS_URBAN,IS_REVOLVER,IS_LOW_DOC,DEFAULT_DATE,DISBURSEMENT_DATE,DISBURSEMENT_AMOUNT,BALANCE_AMOUNT,LOAN_STATUS,CHARGE_OFF_AMOUNT,LOAN_AMOUNT,SBA_APPROVED_AMOUNT,TREASURY_YIELD,CPI_INDEX,GDP,MORTGAGE_30_US_FIXED,UNRATE,INDPRO_INDEX,UMCSENT_INDEX,CSUSHPINSA_INDEX,CP_INDEX,FEDFUNDS_RATE
119522,JOHNSSON REESE FINANCIAL SOLUT,CHICAGO,IL,60622,FIFTH THIRD BANK,IL,0,1994-05-17,1994,36,16,True,0,0,1,True,False,False,NaT,1994-10-31,20000.0,$0.00,True,0.0,20000.0,18000.0,7.080000,148.225,7287.23650,8.380769,6.100000,68.713433,92.266667,79.469083,438.35075,4.201667
119527,COMPUTERLAND OF DOWNERS GROVE,CHICAGO,IL,60515,BMO HARRIS BK NATL ASSOC,IL,0,1994-05-17,1994,84,6,True,0,0,19110,True,False,False,NaT,1994-10-31,250000.0,$0.00,True,0.0,250000.0,200000.0,7.080000,148.225,7287.23650,8.380769,6.100000,68.713433,92.266667,79.469083,438.35075,4.201667
119675,SEARS RETAIL DEALER STORE,SELMA,AL,36701,TRUSTMARK NATIONAL BANK,AL,0,1994-05-19,1994,120,3,False,0,0,1,False,False,False,NaT,1994-07-31,213500.0,$0.00,True,0.0,213500.0,181475.0,7.080000,148.225,7287.23650,8.380769,6.100000,68.713433,92.266667,79.469083,438.35075,4.201667
120580,NICOLE'S,CHICAGO,IL,60622,BANK OF AMERICA NATL ASSOC,IL,0,1994-05-31,1994,84,6,True,0,0,1,True,False,False,NaT,1994-07-31,75000.0,$0.00,True,0.0,75000.0,67500.0,7.080000,148.225,7287.23650,8.380769,6.100000,68.713433,92.266667,79.469083,438.35075,4.201667
121009,"KID'S TIME, INC.",SKOKIE,IL,60618,BANK OF AMERICA NATL ASSOC,IL,624410,1994-06-03,1994,84,1,False,0,0,45499,True,False,False,NaT,1994-10-31,60000.0,$0.00,True,0.0,60000.0,54000.0,7.080000,148.225,7287.23650,8.380769,6.100000,68.713433,92.266667,79.469083,438.35075,4.201667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899159,MARK DEMATTIA INCORPORATED,EWA BEACH,HI,96706,CENTRAL PACIFIC BANK,HI,492210,2014-06-12,2014,60,0,False,0,0,0,True,False,False,NaT,2014-06-12,50000.0,$0.00,True,0.0,50000.0,25000.0,2.540833,236.715,17608.13825,4.168868,6.158333,102.305642,84.125000,164.674667,1947.36300,0.089167
899160,PURSUIT SPORTS MEDICINE LLC,PORTLAND,OR,97232,U.S. BANK NATIONAL ASSOCIATION,OH,621111,2014-06-18,2014,60,1,True,0,1,0,True,False,False,NaT,2014-06-18,165000.0,$0.00,True,0.0,165000.0,82500.0,2.540833,236.715,17608.13825,4.168868,6.158333,102.305642,84.125000,164.674667,1947.36300,0.089167
899161,"Face Reality, Inc., dba Face R",San Leandro,CA,94577,CITY NATIONAL BANK,CA,325412,2014-06-19,2014,84,15,True,3,0,0,True,False,False,NaT,2014-06-19,150000.0,$0.00,True,0.0,150000.0,75000.0,2.540833,236.715,17608.13825,4.168868,6.158333,102.305642,84.125000,164.674667,1947.36300,0.089167
899162,HARDRIVES CONSTRUCTION INC,BILLINGS,MT,59103,BANK OF BRIDGER NATL ASSOC,MT,237310,2014-06-23,2014,6,30,True,0,30,0,False,True,False,NaT,2014-06-23,210000.0,$0.00,True,0.0,210000.0,157500.0,2.540833,236.715,17608.13825,4.168868,6.158333,102.305642,84.125000,164.674667,1947.36300,0.089167


In [147]:
processed_noReduced_ld_df = processed_noReduced_ld_df[processed_noReduced_ld_df['INDUSTRY_ID'] != 0]


In [148]:
processed_row_reduced_ld_df = processed_noReduced_ld_df.copy()

In [149]:
processed_row_reduced_ld_df = processed_row_reduced_ld_df.drop(columns=['CITY', 'BALANCE_AMOUNT'])

In [150]:
processed_row_reduced_ld_df = processed_row_reduced_ld_df.reset_index(drop = True)
processed_row_reduced_ld_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319455 entries, 0 to 319454
Data columns (total 34 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   BORROWER_NAME         319455 non-null  object        
 1   STATE                 319455 non-null  object        
 2   ZIP                   319455 non-null  int64         
 3   BANK                  319455 non-null  object        
 4   BANK_STATE            319455 non-null  object        
 5   INDUSTRY_ID           319455 non-null  int64         
 6   APPROVAL_DATE         319455 non-null  datetime64[ns]
 7   APPROVAL_YEAR         319455 non-null  object        
 8   TERM                  319455 non-null  int64         
 9   EMPLOYEE_COUNT        319455 non-null  int64         
 10  IS_NEW                319455 non-null  object        
 11  JOBS_CREATED_COUNT    319455 non-null  int64         
 12  JOBS_RETAINED_COUNT   319455 non-null  int64         
 13 

In [151]:
processed_row_reduced_ld_df['EXPOSURE'] = processed_row_reduced_ld_df['DISBURSEMENT_AMOUNT'] -  processed_row_reduced_ld_df['SBA_APPROVED_AMOUNT']
processed_row_reduced_ld_df['PERCENTAGE_EXPOSURE'] = processed_row_reduced_ld_df['EXPOSURE']/processed_row_reduced_ld_df ['DISBURSEMENT_AMOUNT']

In [152]:
processed_row_reduced_ld_df.head()

,BORROWER_NAME,STATE,ZIP,BANK,BANK_STATE,INDUSTRY_ID,APPROVAL_DATE,APPROVAL_YEAR,TERM,EMPLOYEE_COUNT,IS_NEW,JOBS_CREATED_COUNT,JOBS_RETAINED_COUNT,FRANCHISE_CODE,IS_URBAN,IS_REVOLVER,IS_LOW_DOC,DEFAULT_DATE,DISBURSEMENT_DATE,DISBURSEMENT_AMOUNT,LOAN_STATUS,CHARGE_OFF_AMOUNT,LOAN_AMOUNT,SBA_APPROVED_AMOUNT,TREASURY_YIELD,CPI_INDEX,GDP,MORTGAGE_30_US_FIXED,UNRATE,INDPRO_INDEX,UMCSENT_INDEX,CSUSHPINSA_INDEX,CP_INDEX,FEDFUNDS_RATE,EXPOSURE,PERCENTAGE_EXPOSURE
0,"KID'S TIME, INC.",IL,60618,BANK OF AMERICA NATL ASSOC,IL,624410,1994-06-03,1994,84,1,False,0,0,45499,True,False,False,NaT,1994-10-31,60000.0,True,0.0,60000.0,54000.0,7.080000,148.225000,7287.23650,8.380769,6.100000,68.713433,92.266667,79.469083,438.35075,4.201667,6000.0,0.10
1,"DERHAM & REEVES, EA'S",CA,94538,WELLS FARGO BANK NATL ASSOC,SD,541213,1995-11-24,1996,84,2,True,0,0,1,True,False,False,NaT,1996-01-31,24000.0,True,0.0,24000.0,19200.0,6.438333,156.858333,8073.12175,7.806154,5.408333,75.165983,93.608333,82.808500,541.42850,5.298333,4800.0,0.20
2,TEMPORARY LIVING OF COLUMBUS,OH,43231,JPMORGAN CHASE BANK NATL ASSOC,IL,721310,1995-12-15,1996,84,1,False,0,0,1,True,False,False,NaT,1996-01-31,140000.0,True,0.0,140000.0,105000.0,6.438333,156.858333,8073.12175,7.806154,5.408333,75.165983,93.608333,82.808500,541.42850,5.298333,35000.0,0.25
3,BALOONS AND BEARS,CA,94598,WELLS FARGO BANK NATL ASSOC,SD,453220,1996-01-11,1996,84,1,False,0,0,8047,True,False,False,2003-04-15,1996-04-30,46200.0,False,19329.0,46200.0,36960.0,6.438333,156.858333,8073.12175,7.806154,5.408333,75.165983,93.608333,82.808500,541.42850,5.298333,9240.0,0.20
4,BALLOONS AND BEARS,CA,94566,WELLS FARGO BANK NATL ASSOC,SD,453220,1996-01-11,1996,84,3,False,0,0,8047,True,False,False,NaT,1996-04-30,50000.0,True,0.0,50000.0,40000.0,6.438333,156.858333,8073.12175,7.806154,5.408333,75.165983,93.608333,82.808500,541.42850,5.298333,10000.0,0.20
